# CS4661 (Intro to Data Science) - Shelter Animal Outcomes
# Author: Group #3
## Dataset: https://www.kaggle.com/c/shelter-animal-outcomes

In [1]:
"""
Preprocesses the shelter-animal-outcomes (sao) dataset

Parameters:

Returns:

"""

### Importing Libraries

In [2]:
import numpy as np
import pandas as pd

### Reading the sao dataset

In [3]:
df = pd.read_csv('../../data/train.csv') # stores the sao testing dataset 
df[:10] # displays first 10 rows

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White


In [4]:
feature_cols = list(df.columns) # stores each column name in a list
feature_cols # displays all column headers

['AnimalID',
 'Name',
 'DateTime',
 'OutcomeType',
 'OutcomeSubtype',
 'AnimalType',
 'SexuponOutcome',
 'AgeuponOutcome',
 'Breed',
 'Color']

## Change AnimalType to Two Columns

In [5]:
df = pd.concat([df, df['AnimalType'].str.get_dummies()], axis=1)
df[:5]

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Cat,Dog
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,0,1
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,1,0
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,0,1
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,1,0
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,0,1


## Change SexuponOutcome to multiple columns

In [6]:
# Replace all NaN's with Unknown
df.SexuponOutcome.fillna('Unknown', inplace=True)

# Print all the unique Sex Types
print(set(df.SexuponOutcome)) # note: Unknown sex probably doesn't matter

df = pd.concat([df, df['SexuponOutcome'].str.get_dummies(sep=' ')], axis=1)
df[:5]

{'Intact Female', 'Intact Male', 'Spayed Female', 'Unknown', 'Neutered Male'}


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Cat,Dog,Female,Intact,Male,Neutered,Spayed,Unknown
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,0,1,0,0,1,1,0,0
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,1,0,1,0,0,0,1,0
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,0,1,0,0,1,1,0,0
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,1,0,0,1,1,0,0,0
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,0,1,0,0,1,1,0,0


## Creating an age function for AgeuponOutcome

In [7]:
age_weights = {'year': 365, 'years': 365, 'month': 30, 'months': 30, 'week': 7, 'weeks': 7, 'day': 1, 'days': 1, 'nan': 0}

def age_conversion(age):
    age = age.split(' ')
    
    multiplier = age[0]
    weight = age[1]
    
    return int(multiplier) * int(age_weights[weight])

In [8]:
# Replace all NaN's with Unknown
df.AgeuponOutcome.fillna('0 nan', inplace=True)

age = list(df.AgeuponOutcome)
age_conv = list(map(age_conversion, age))

age = pd.Series(age_conv)
df['Age'] = age

df[:5]

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Cat,Dog,Female,Intact,Male,Neutered,Spayed,Unknown,Age
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,0,1,0,0,1,1,0,0,365
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,1,0,1,0,0,0,1,0,365
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,0,1,0,0,1,1,0,0,730
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,1,0,0,1,1,0,0,0,21
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,0,1,0,0,1,1,0,0,730


## Change Color Column to Multiple Color Columns

In [9]:
colors = [c.replace(' ', '/') for c in list(df.Color)] # replace all spaces in Color with '/'
df.Color = colors

# pd.get_dummies(df["Color"])
# >>> Series(['a|b', 'a', 'a|c']).str.get_dummies()
df = pd.concat([df, df['Color'].str.get_dummies(sep='/')], axis=1)

df[:5]

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,...,Smoke,Tabby,Tan,Tick,Tiger,Torbie,Tortie,Tricolor,White,Yellow
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,...,0,0,0,0,0,0,0,0,1,0
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream/Tabby,...,0,1,0,0,0,0,0,0,0,0
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,...,0,0,0,0,0,0,0,0,1,0
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue/Cream,...,0,0,0,0,0,0,0,0,0,0
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,...,0,0,1,0,0,0,0,0,0,0


## Change Breed Column to Multiple Breed Columns

In [10]:
breeds = ['Mixed' if 'Mix' in b else 'Purebred' for b in list(df.Breed)]
df.Breed = breeds

# pd.get_dummies(df["Color"])
# >>> Series(['a|b', 'a', 'a|c']).str.get_dummies()
df = pd.concat([df, df['Breed'].str.get_dummies(sep='/')], axis=1)

## Remove all the unneeded columns

In [11]:
junk_cols = ['AnimalID', 'OutcomeSubtype', 'Name', 'DateTime', 'Color', 'Unknown', 'AgeuponOutcome', 'AnimalType', 'SexuponOutcome', 'Breed'] # useless columns
df = df.drop(junk_cols, axis=1) # drop the junk cols from the dataset
df[:10] # displays first 10 rows of changed df

,OutcomeType,Cat,Dog,Female,Intact,Male,Neutered,Spayed,Age,Agouti,...,Tan,Tick,Tiger,Torbie,Tortie,Tricolor,White,Yellow,Mixed,Purebred
0,Return_to_owner,0,1,0,0,1,1,0,365,0,...,0,0,0,0,0,0,1,0,1,0
1,Euthanasia,1,0,1,0,0,0,1,365,0,...,0,0,0,0,0,0,0,0,1,0
2,Adoption,0,1,0,0,1,1,0,730,0,...,0,0,0,0,0,0,1,0,1,0
3,Transfer,1,0,0,1,1,0,0,21,0,...,0,0,0,0,0,0,0,0,1,0
4,Transfer,0,1,0,0,1,1,0,730,0,...,1,0,0,0,0,0,0,0,0,1
5,Transfer,0,1,1,1,0,0,0,30,0,...,1,0,0,0,0,0,0,0,0,1
6,Transfer,1,0,0,1,1,0,0,21,0,...,0,0,0,0,0,0,0,0,1,0
7,Transfer,1,0,0,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0,1,0
8,Adoption,0,1,1,0,0,0,1,150,0,...,0,0,0,0,0,0,1,0,1,0
9,Adoption,0,1,1,0,0,0,1,365,0,...,0,0,0,0,0,0,1,0,0,1


## New Working Dataset

In [12]:
df[::5] # Final dataset

,OutcomeType,Cat,Dog,Female,Intact,Male,Neutered,Spayed,Age,Agouti,...,Tan,Tick,Tiger,Torbie,Tortie,Tricolor,White,Yellow,Mixed,Purebred
0,Return_to_owner,0,1,0,0,1,1,0,365,0,...,0,0,0,0,0,0,1,0,1,0
5,Transfer,0,1,1,1,0,0,0,30,0,...,1,0,0,0,0,0,0,0,0,1
10,Transfer,1,0,0,0,0,0,0,730,0,...,0,0,0,0,0,0,0,0,1,0
15,Adoption,1,0,0,0,1,1,0,90,0,...,0,0,0,0,0,0,1,0,1,0
20,Transfer,0,1,0,1,1,0,0,60,0,...,0,0,0,0,0,0,1,0,1,0
25,Transfer,1,0,1,1,0,0,0,30,0,...,0,0,0,0,0,0,0,0,1,0
30,Euthanasia,1,0,1,0,0,0,1,1095,0,...,0,0,0,0,0,0,0,0,1,0
35,Euthanasia,0,1,0,1,1,0,0,1095,0,...,0,0,0,0,0,0,0,1,1,0
40,Adoption,1,0,1,0,0,0,1,2555,0,...,0,0,0,0,0,0,0,0,1,0
45,Return_to_owner,0,1,0,0,1,1,0,4380,0,...,0,0,0,0,0,1,0,0,0,1


## Write out new dataset to csv file

In [13]:
df.to_csv('../../data/train_V2.csv')